In [1]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import time
import logging
import json
import lxml

In [2]:
logging.basicConfig(filename='LK1.log', level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')
colour=[]
size=[]
width=[]
brand=[]
model=[]
price=[]
type=[]
count=[]
rating=[]
no_rating=[]
link=[]
name=[]
offer=[]
i=1

In [ ]:
while True:
    url= 'https://api-gateway.juno.lenskart.com/v2/products/category/3363?page-size=15&page={}'.format(i)
    response=requests.get(url)
    data = json.loads(response.text)
    p = data['result']['product_list']
    if len(p)==0:
        logging.info("Page "+str(i)+" not found")
        break
    else:
        logging.info("Page "+str(i)+" found")
        
    for product in p:
        check=[]
        try:
            colour.append(product['color'])
        except:
            colour.append("")
            check.append("colour")
        try:
            size.append(product['size'])
        except:
            size.append("")
            check.append("size")
        try:
            width.append(product['width'])
        except:
            width.append("")
            check.append("width")

        brand.append(product['brand_name'])
        model.append(product['model_name'])
        price.append(product['prices'][1]['price'])
        type.append(product['classification'])
        try:
            count.append(product['purchaseCount'])
        except:
            count.append("")
            check.append("count")
        try:
            rating.append(product['avgRating'])
        except:
            rating.append("")
            check.append("rating")
        try:
            no_rating.append(product['totalNoOfRatings'])
        except:
            no_rating.append(0)
        link.append(product['product_url'])
        c=" ".join(check)
        logging.info(str(i)+" "+c+ " Not Found" )
    i+=1

In [ ]:
m=1
for k in link:
    res=requests.get(url=k)
    soup=BeautifulSoup(res.content,'lxml')
    n=soup.find('h1',class_='Title--mnzriy sVVvQ')
    if n is None:
        n=soup.find('h1',class_='basicDetailsstyles__Title-sc-2nv0zt-1 eIDqJd')
    try:
        name.append(n.text)
        
    except:
        name.append("")
        logging.info(str(m) + " Name not found")
    o=soup.find('div',class_='OfferDetailsText2--17bl9a4 coTxPj')
    if o is not None:
        offer.append(o.text)
    else:
        offer.append("")
        logging.info(str(m) + " Offer not found")
    m+=1

In [17]:
data = {'Name': name,
        'Brand': brand,
        'Type':type,
        'Size': size,
        'Frame Width': width,
        'Colour':colour,
        'Model No.': model,
        'Price':price,
        'offer':offer,
        'Units Sold':count,
        'Average Rating':rating,
        'Number of Ratings':no_rating}

df = pd.DataFrame(data)
excel_file = 'Eyeglasses.xlsx'
df.to_excel(excel_file, index=False)